In [1]:
import requests

import pandas as pd
import datetime

import os

In [2]:
# Get symbols that user is tracking
names=["AAVE","MakerDAO"]

In [3]:
def lookup(name):
    """Look up TVL for name"""

    # Contact API
    try:
        url = f"https://api.llama.fi/protocol/{name}"
        response = requests.get(url)
        response.raise_for_status()
    except requests.RequestException:
        return None

    # Parse response
    try:
        quote = response.json()
        
        # Delete uneeded data
        chainTvls=quote["chainTvls"]
        for chain in chainTvls:
            chainTvls[chain]=chainTvls[chain]["tvl"]
                
        return {
            "name": quote["name"],
            "chainTvls":chainTvls,
            "tvl":quote["tvl"]
        }
    except (KeyError, TypeError, ValueError):
        return None

In [4]:
# Put all data into single dataframe
df_detailed = pd.DataFrame()

for name in names:
    data=lookup(name)

    for chain,dateTvl_dict in data["chainTvls"].items():
        current_df=pd.DataFrame.from_records(dateTvl_dict)
        current_df["protocol"]=data["name"]
        current_df["chain"]=chain
        df_detailed=pd.concat([df_detailed,current_df])
    
    current_df=pd.DataFrame.from_records(data["tvl"])
    current_df["protocol"]=data["name"]
    current_df["chain"]="actual tvl"
    df_detailed=pd.concat([df_detailed,current_df])

In [5]:
# Change column from int64 to datetime
df_detailed['date'] = pd.to_datetime(df_detailed['date'], unit='s')

# Sorting the index according to the protocol and chain
df_detailed=df_detailed.set_index(["protocol","chain","date"]).sort_index()

In [6]:
# Create a df with just the overview of the protocol
df_summary=df_detailed.loc[(slice(None),["pool2","staking","actual tvl","borrowed"]),:]

df_summary=df_summary.sort_index()

In [7]:
# Export data to excel in the same directory
dir_path = os.getcwd()
with pd.ExcelWriter(f"{dir_path}/tvl.xlsx") as writer: 
    df_summary.to_excel(writer,sheet_name="Summary")
    df_detailed.to_excel(writer,sheet_name="Detailed")